In [ ]:
! pip install snscrape

In [ ]:
import numpy as np

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [ ]:
query_test = "web scraping "
for tweet in sntwitter.TwitterSearchScraper(query_test).get_items():
    items = vars(tweet)
    break

In [ ]:
from datetime import date, timedelta

start_date = date(2022, 11, 1) 
end_date = date(2023, 4, 17)    # perhaps date.now()

delta = end_date - start_date   # returns timedelta

days = []

for i in range(delta.days + 1):
    day = str(start_date + timedelta(days=i))
    days.append(day)

In [ ]:
def scrape_tweets(query, limit):
    tweets = []
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        if len(tweets) == limit:
            break
        else:
            tweets.append([tweet.date, tweet.user.username, tweet.rawContent,
                        tweet.retweetCount, tweet.likeCount, tweet.replyCount,
                        tweet.user.followersCount])

    # df = pd.DataFrame(tweets, 
    #                   columns=['Date', 'User', 'Tweet', 'retweetCount', 
    #                            'likeCount', 'replyCount', 'followersCount'])
    return tweets

In [ ]:
tweets = []
for i, day in enumerate(days):
    if i < 10:
        first_date = day
        second_date = days[i+1]
        time = " since:" + first_date + " until:" + second_date
        query = keywords_random + lang + time
        print(query)
        tweet = scrape_tweets(query, 100)
        tweets.extend(tweet)
    else:
        break

In [ ]:
def scrape_daily(keywords, days, limit, language):
    tweets = []
    for i, day in enumerate(days):
        if i < len(days) - 1:
            first_date = day
            second_date = days[i+1]
            time = " since:" + first_date + " until:" + second_date
            query = keywords + language + time
            try:
                tweet = scrape_tweets(query, limit)
                print(time)
            except:
                return tweets
            tweets.extend(tweet)
    
    # df = pd.DataFrame(tweets, 
    #                   columns=['Date', 'User', 'Tweet', 'retweetCount', 
    #                            'likeCount', 'replyCount', 'followersCount'])
    
    return tweets

In [ ]:
keywords_random = "(the) -abortion -miscarriage -music -movie"
keywords_abortion = "(abortion OR miscarriage) -music -movie"
keywords_music = "(music OR movie) -abortion -miscarriage"

lang = " lang:en"
daily_limit = 30

In [ ]:
tweets_random = scrape_daily(keywords_random, days, daily_limit, lang)

In [ ]:
df_random = pd.DataFrame(tweets_random, 
                      columns=['Date', 'User', 'Tweet', 'retweetCount', 
                               'likeCount', 'replyCount', 'followersCount'])

In [ ]:
df_random.to_csv('random_daily.csv')

In [ ]:
tweets_music = scrape_daily(keywords_music, days, daily_limit, lang)

In [ ]:
df_music = pd.DataFrame(tweets_music, 
                      columns=['Date', 'User', 'Tweet', 'retweetCount', 
                               'likeCount', 'replyCount', 'followersCount'])

In [ ]:
tweets_abortion = scrape_daily(keywords_abortion, days, daily_limit, lang)

In [ ]:
df_abortion = pd.DataFrame(tweets_abortion, 
                      columns=['Date', 'User', 'Tweet', 'retweetCount', 
                               'likeCount', 'replyCount', 'followersCount'])

In [ ]:
! pip install vaderSentiment

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
def calculate_intensity(df):
    results = []
    for tweet in df['Tweet']:
        vs = analyzer.polarity_scores(tweet)
        results.append(vs)
    
    df_results = pd.DataFrame(results)
    df_concat = pd.concat([df, df_results], axis=1)
    df_concat['engagement'] = df_concat['retweetCount'] + df_concat['likeCount'] + df_concat['replyCount']
    df_concat['absolute intensity'] = abs(df_concat['compound'])

    return df_concat

In [ ]:
df_random = calculate_intensity(df_random)

In [ ]:
df_abortion =  calculate_intensity(df_abortion)

In [ ]:
df_music = calculate_intensity(df_music)

In [ ]:
df_random.to_csv('random.csv')
df_abortion.to_csv('abortion.csv')
df_music.to_csv('music.csv')